# Beta expanded ensemble

In [1]:
import feasst as fst

mc = fst.MonteCarlo()
mc.set(fst.lennard_jones())
mc.set(fst.MakeThermoParams(fst.args({"beta": "1.2", "chemical_potential": "1."})))
beta_min = 0.8
beta_max = 1.2
beta_num = 5
delta_beta = str((beta_max - beta_min)/(beta_num-1))
mc.set(fst.MakeFlatHistogram(
    fst.MakeMacrostateBeta(
        fst.Histogram(fst.args({"width": delta_beta, "max": str(beta_max), "min": str(beta_min)}))),
    fst.MakeWLTM(fst.args({"collect_flatness": "18", "min_flatness": "22", "min_sweeps": "10"}))))
mc.add(fst.MakeTrialTranslate(fst.args({"weight": "1.", "tunable_param": "1."})))
fst.SeekNumParticles(10).with_trial_add().with_metropolis().run(mc)
mc.add(fst.MakeTrialBeta(fst.args({"fixed_beta_change": delta_beta})))
steps_per = str(int(1e4))
mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "lj_beta"})))
mc.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per": steps_per})))
mc.add(fst.MakeCriteriaUpdater(fst.args({"steps_per": steps_per})))
mc.add(fst.MakeCriteriaWriter(fst.args({"steps_per": steps_per, "file_name": "lj_beta_crit.txt",
    "file_name_append_phase": "true"})))
mc.add(fst.MakeEnergy(fst.args({
    "file_name": "lj_beta_energy",
    "file_name_append_phase": "true",
    "steps_per_update": "1",
    "steps_per_write": steps_per,
    "multistate": "true"})))
mc.attempt(int(1e6))